In [ ]:
import os
import pandas as pd
from relm.histogram import Histogram
from relm.mechanisms import SmallDB
import numpy as np
from itertools import product
import scipy.sparse as sps

In [ ]:
fp = '20200811_QLD_dummy_dataset_individual_v2.xlsx'
df = pd.read_excel(fp)
# df.drop(["NOTF_ID", "LGA", "HHS"] + list(df.columns[12:]), axis=1, inplace=True)
df.drop(list(df.columns[1:6]) + list(df.columns[7:]), axis=1, inplace=True)
hist = Histogram(df)
real_database = hist.get_db()

In [ ]:
from itertools import product
# _cols = ["AGEGRP5", "SEX", "INDIG_STATUS"]
queries = []

# # this creates the queries for:
# # df.groupby(["AGEGRP5", "SEX", "INDIG_STATUS", col]).count()
# # where col is in ["HOSPITALISED", "VENTILATED", "ICU", "DIED_OF_CONDITION"]
# for col in ["HOSPITALISED", "VENTILATED", "ICU", "DIED_OF_CONDITION"]:
#     cols = _cols + [col,]
#     vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
#     queries.extend(dict(zip(cols, val)) for val in vals)

# # this creates the queries for:
# # df.groupby(["ONSET_DATE", "AGEGRP5", "INDIG_STATUS", "SEX"]).count()
# cols = ["ONSET_DATE", "AGEGRP5", "INDIG_STATUS", "SEX"]
# vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
# queries.extend(dict(zip(cols, val)) for val in vals)

# this creates the queries for:
# df.groupby(["ONSET_DATE", "POSTCODE"]).count()
cols = ["ONSET_DATE", "POSTCODE"]
vals = product(*[list(hist.column_sets[hist.column_dict[c]]) for c in cols])
queries.extend(dict(zip(cols, val)) for val in vals)

queries = sps.vstack([hist.get_query_vector(q) for q in queries])

In [ ]:
smalldb = SmallDB(epsilon=0.001, data=real_database, alpha=0.1)

In [ ]:
synthetic_database = smalldb.release(queries)

In [ ]:
real_responses = (queries @ real_database)/real_database.sum()
synthetic_responses = (queries @ synthetic_database) / synthetic_database.sum()

In [ ]:
print(real_responses)
print(synthetic_responses)

In [ ]:
queries